# Тестирование вероятностной тематической модели


In [2]:
import sys
import os
current_dir = os.path.abspath('')
project_root = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.insert(0, project_root)

In [3]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
import time
from LDA import LDA
from sklearn.decomposition import LatentDirichletAllocation

## Подготовка данных

In [4]:
categories = ['soc.religion.christian','comp.graphics', 'sci.med','rec.motorcycles']
X,y = fetch_20newsgroups(categories=categories,return_X_y=True)
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.2, random_state=2025)

## Тестирование латентного размещения Дирихле

In [4]:
lda=LDA(num_themes=4, alpha=1.5, beta=1.5)

In [6]:
start = time.time()
phi_train,theta_train=lda.fit_predict(train_text=X_train,max_steps=500)
end = time.time()
print("Время выполнения реализованного алгоритма:",
      (end-start) * 10**3, "ms")
lda_etalon=LatentDirichletAllocation(n_components=4)
start = time.time()
X_t=lda.vectorizer.transform(X_train)
etalon_matrix=lda_etalon.fit_transform(X_t)
end = time.time()
print("Время выполнения эталонного алгоритма:",
      (end-start) * 10**3, "ms")

Время выполнения реализованного алгоритма: 1490396.1744308472 ms
Время выполнения эталонного алгоритма: 4752.091646194458 ms


## Решение задачи классификации

Задача классификации в данном случае требует ручной разметки подходящих столбцов


In [7]:
y_pred_train=np.argmax(theta_train,axis=0)
pd.crosstab(y_pred_train,y_train)

col_0,0,1,2,3
row_0,,,,
0,5,3,279,4
1,14,7,149,463
2,417,3,15,9
3,24,470,30,8


In [8]:
def get_accuracy(matches_dict,y_pred,y_real):#Расчёт accuracy
    trues=0
    for ind_pred, ind_real in matches_dict.items():
        trues+=np.sum(np.isin(np.where(y_pred==ind_pred), np.where(y_real==ind_real)))
    return trues/len(y_real)

In [11]:
matches={
    0:2,
    1:3,
    2:0,
    3:1
}
print("Точность на тренировочных данных:",round(get_accuracy(matches,y_pred_train,y_train),3))

Точность на тренировочных данных: 0.857


In [12]:
theta_test=lda.predict(X_test,max_steps=100)

In [13]:
y_pred_test=np.argmax(theta_test,axis=0)
pd.crosstab(y_test,y_pred_test)

col_0,0,1,2,3
row_0,,,,
0,6,32,63,23
1,6,25,11,73
2,53,40,10,18
3,8,85,6,16


In [14]:
print("Точность на тренировочных данных:",round(get_accuracy(matches,y_pred_test,y_test),3))

Точность на тренировочных данных: 0.577


In [15]:
y_etalon_pred_train=np.argmax(etalon_matrix,axis=1)
pd.crosstab(y_etalon_pred_train,y_train)

col_0,0,1,2,3
row_0,,,,
0,37,31,195,468
1,95,34,167,7
2,33,309,56,2
3,295,109,55,7


In [16]:
matches_etalon={
    0:3,
    1:2,
    2:1,
    3:0
}
print("Точность на тренировочных данных:",round(get_accuracy(matches_etalon,y_etalon_pred_train,y_train),3))

Точность на тренировочных данных: 0.652


## Расчёт когерентностей тем

In [5]:
from utils.metrics import get_coherence_score

Рассчитаем когерентности тем на всём датасете

In [6]:
lda_test=LDA(num_themes=4, alpha=1.5, beta=1.5)
_,_=lda_test.fit_predict(train_text=X_train,max_steps=500)
get_coherence_score(lda_test.vectorizer, lda_test.phi, X, metric='u_mass')#По реализованному алгоритму


Когерентность тем (u_mass): -0.3728


In [9]:
lda_test_etalon=LatentDirichletAllocation(n_components=4)
X_t=lda_test.vectorizer.transform(X)
lda_test_etalon.fit_transform(X_t)
etalon_phi=lda_test_etalon.components_/lda_test_etalon.components_.sum(axis=1, keepdims=True)

In [141]:
get_coherence_score(lda.vectorizer, etalon_phi, X_train, metric='u_mass')#По эталонному алгоритму

Когерентность тем (u_mass): -0.3968
